In [1]:
import torch
from torchvision import datasets, transforms
from torch import tensor

In [2]:
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [5]:
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
valid_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

In [6]:
print(len(valid_dataset))
print(len(train_dataset))

10000
60000


In [10]:
def init_params(size, std = 1.0):
    (torch.randn(size) * std).requires_grad_()

In [7]:
from torch.utils.data import DataLoader

train_dl = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
valid_dl = DataLoader(dataset=valid_dataset, batch_size=1000, shuffle=False)

In [8]:
from PIL import Image

img, label = valid_dataset[1000]
img_pil = transforms.ToPILImage()(img)
img_pil.show()


In [11]:
weights = init_params((28*28, 1))
bias = init_params(1)

In [12]:
def mnist_loss(predictions, targets):
    return torch.where(targets==1, 1-predictions, predictions).mean()

In [13]:
import os
import torch
from torchvision import datasets, transforms

dataset_path = './data/MNIST'

for root, dirs, files in os.walk(dataset_path):
    print("Root:", root)
    print("Subdirectories:", dirs)
    print("Files:", files)

Root: ./data/MNIST
Subdirectories: ['raw']
Files: []
Root: ./data/MNIST/raw
Subdirectories: []
Files: ['t10k-images-idx3-ubyte', 't10k-labels-idx1-ubyte', 'train-images-idx3-ubyte', 't10k-images-idx3-ubyte.gz', 'train-images-idx3-ubyte.gz', 'train-labels-idx1-ubyte.gz', 'train-labels-idx1-ubyte', 't10k-labels-idx1-ubyte.gz']


In [14]:
from collections import Counter

labels = [label for _, label in train_dataset]

label_counts = Counter(labels)

for digit, count in sorted(label_counts.items()):
    print(f"Digit {digit}: {count} images")

Digit 0: 5923 images
Digit 1: 6742 images
Digit 2: 5958 images
Digit 3: 6131 images
Digit 4: 5842 images
Digit 5: 5421 images
Digit 6: 5918 images
Digit 7: 6265 images
Digit 8: 5851 images
Digit 9: 5949 images


In [15]:
digit_tensors = {digit: [] for digit in range(10)}

for img, label in train_dataset:
    digit_tensors[label].append(img)
#print(digit_tensors[0])
digit_tensors = {digit: torch.stack(images).squeeze(1).view(-1, 28, 28) for digit, images in digit_tensors.items()}
#print(digit_tensors[0])
for digit,tensor in digit_tensors.items():
    print(f"Digit {digit} tensor shape: {tensor.shape}")

print(digit_tensors.keys())

#zero_tensors = [tensor(Image.open(o)) for o in zer0s]

Digit 0 tensor shape: torch.Size([5923, 28, 28])
Digit 1 tensor shape: torch.Size([6742, 28, 28])
Digit 2 tensor shape: torch.Size([5958, 28, 28])
Digit 3 tensor shape: torch.Size([6131, 28, 28])
Digit 4 tensor shape: torch.Size([5842, 28, 28])
Digit 5 tensor shape: torch.Size([5421, 28, 28])
Digit 6 tensor shape: torch.Size([5918, 28, 28])
Digit 7 tensor shape: torch.Size([6265, 28, 28])
Digit 8 tensor shape: torch.Size([5851, 28, 28])
Digit 9 tensor shape: torch.Size([5949, 28, 28])
dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


In [16]:
for digit in digit_tensors:
    meanie = digit_tensors[digit].mean(0)
    #print("Mean image: " + meanie)
    img_p = transforms.ToPILImage()(meanie)
    img_p.show()

In [19]:
valid_digit_tensors = {digit: [] for digit in range(10)}

for img, label in valid_dataset:
    valid_digit_tensors[label].append(img)

valid_digit_tensors = {digit: torch.stack(images).squeeze(1).view(-1, 28, 28) for digit, images in valid_digit_tensors.items()}

for digit,tensor in valid_digit_tensors.items():
    print(f"Digit {digit} tensor shape: {tensor.shape}")

Digit 0 tensor shape: torch.Size([980, 28, 28])
Digit 1 tensor shape: torch.Size([1135, 28, 28])
Digit 2 tensor shape: torch.Size([1032, 28, 28])
Digit 3 tensor shape: torch.Size([1010, 28, 28])
Digit 4 tensor shape: torch.Size([982, 28, 28])
Digit 5 tensor shape: torch.Size([892, 28, 28])
Digit 6 tensor shape: torch.Size([958, 28, 28])
Digit 7 tensor shape: torch.Size([1028, 28, 28])
Digit 8 tensor shape: torch.Size([974, 28, 28])
Digit 9 tensor shape: torch.Size([1009, 28, 28])


In [20]:
for digit in valid_digit_tensors:
    meanie = valid_digit_tensors[digit].mean(0)
    #print("Mean image: " + meanie)
    img_p = transforms.ToPILImage()(meanie)
    img_p.show()

In [21]:
def calc_grad(xb, yb, model):
    preds = model(xb)
    loss = mnist_loss(preds, yb)
    loss.backward()

In [27]:
def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    correct = (preds > 0.5) == yb
    return correct.float().mean()

def validate_epoch(model):
    accs = [batch_accuracy(model(xb.view(xb.size(0), -1)), yb) for xb, yb in valid_dl]
    return round(torch.stack(accs).mean().item(), 4)

In [23]:
import torch.nn as nn
linear_model = nn.Linear(28*28, 1)

In [28]:
w,b = linear_model.parameters()
w.shape,b.shape

(torch.Size([1, 784]), torch.Size([1]))

In [24]:
class BasicOptim:
    def __init__(self, params, lr):
        self.params, self.lr = list(params), lr

    def step(self, *args, **kwargs):
        for p in self.params: p.data -= p.grad.data * self.lr
    
    def zero_grad(self, *args, **kwargs):
        for p in self.params: p.grad = None

In [29]:
opt = BasicOptim(linear_model.parameters(), 1.)

def train_epoch(model):
    for xb,yb in train_dl:
        xb = xb.view(xb.size(0), -1)
        calc_grad(xb, yb, model)
        opt.step()
        opt.zero_grad()

In [31]:
def train_model(model, epochs):
    for i in range(epochs):
        train_epoch(model)
        print(validate_epoch(model), end = '')

In [32]:
train_model(linear_model, 20)

0.0980.0980.0980.0980.0980.0980.0980.0980.0980.0980.0980.0980.0980.0980.0980.0980.0980.0980.0980.098

In [34]:
simple_net = nn.Sequential(
    nn.Linear(28*28, 30),
    nn.ReLU(),
    nn.Linear(30, 10)
)

In [56]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(simple_net.parameters(), lr=0.01)

def train_model2(epochs):
    for epoch in range(epochs):
        simple_net.train()
        train_loss = 0
        for xb,yb in train_dl:
            xb = xb.view(xb.size(0), -1)
            preds = simple_net(xb)
            loss = criterion(preds, yb)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            train_loss += loss.item()

        simple_net.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for xb, yb in valid_dl:
                xb = xb.view(xb.size(0), -1)
                preds = simple_net(xb)
                val_loss += criterion(preds, yb).item()
                predicted = preds.argmax(dim = 1)
                correct += (predicted == yb).sum().item()
                total += yb.size(0)

        train_loss /= len(train_dl)
        val_loss /= len(valid_dl)
        val_accuracy = correct / total

        print(f"Epoch {epoch+1}/{epochs}")
        print(f"  Train Loss: {train_loss:.4f}")
        print(f"  Val Loss: {val_loss:4f}, Val Accuracy: {val_accuracy:.4f}")


In [57]:
train_model2(50)

Epoch 1/50
  Train Loss: 0.0667
  Val Loss: 0.111105, Val Accuracy: 0.9681
Epoch 2/50
  Train Loss: 0.0661
  Val Loss: 0.111100, Val Accuracy: 0.9678
Epoch 3/50
  Train Loss: 0.0648
  Val Loss: 0.108463, Val Accuracy: 0.9681
Epoch 4/50
  Train Loss: 0.0641
  Val Loss: 0.111206, Val Accuracy: 0.9676
Epoch 5/50
  Train Loss: 0.0632
  Val Loss: 0.107808, Val Accuracy: 0.9683
Epoch 6/50
  Train Loss: 0.0624
  Val Loss: 0.108938, Val Accuracy: 0.9671
Epoch 7/50
  Train Loss: 0.0614
  Val Loss: 0.108378, Val Accuracy: 0.9689
Epoch 8/50
  Train Loss: 0.0606
  Val Loss: 0.109501, Val Accuracy: 0.9682
Epoch 9/50
  Train Loss: 0.0600
  Val Loss: 0.109362, Val Accuracy: 0.9686
Epoch 10/50
  Train Loss: 0.0593
  Val Loss: 0.108578, Val Accuracy: 0.9685
Epoch 11/50
  Train Loss: 0.0583
  Val Loss: 0.109329, Val Accuracy: 0.9690
Epoch 12/50
  Train Loss: 0.0575
  Val Loss: 0.113157, Val Accuracy: 0.9666
Epoch 13/50
  Train Loss: 0.0570
  Val Loss: 0.109111, Val Accuracy: 0.9681
Epoch 14/50
  Train L

In [ ]:
simple_net.eval()

image_path = "three_img.png"
image = Image.open(image_path).convert('L')

testing_transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

image_tensor = testing_transform(image).unsqueeze(0)
# test_img_pil = transforms.ToPILImage()(image_tensor)
# test_img_pil.show()
image_tensor = image_tensor.view(image_tensor.size(0), -1)

with torch.no_grad():
    predictions = simple_net(image_tensor)
    predicted_label = predictions.argmax(dim=1).item()

print(f"Predicted Label: {predicted_label}")


Predicted Label: 1
